In [241]:
import pandas as pd 
import numpy as np                    

In [242]:
# Data Load 
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [243]:
df_train.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


In [244]:
# missing Value
def missing_vaule(df):
    df['Episode_Length_minutes']=df['Episode_Length_minutes'].fillna(df['Episode_Length_minutes'].mean())
    df['Guest_Popularity_percentage']=df['Guest_Popularity_percentage'].fillna(df['Guest_Popularity_percentage'].mean())
    df['Number_of_Ads']=df['Number_of_Ads'].fillna(df_train['Number_of_Ads'].mean())
   
        
missing_vaule(df_train)
missing_vaule(df_test)

In [245]:
# Unnecessary colmun delete

drop_col=['Podcast_Name','Episode_Title']
df_train.drop(drop_col,inplace=True,axis=1)
df_test.drop(drop_col,inplace=True,axis=1)

# Encodeing 

In [246]:
# Encodeing Day Colmuns

from sklearn.preprocessing import OneHotEncoder

# Initialize encoder to handle unknown categories and drop first category
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# Fit on training data and transform both train and test
encode_day = encoder.fit_transform(df_train[['Publication_Day']])
encode_day_t = encoder.transform(df_test[['Publication_Day']])  # Note: Make sure column name matches

# Create DataFrames with feature names
encoded_df = pd.DataFrame(encode_day, 
                         columns=encoder.get_feature_names_out(["Publication_Day"]))
encoded_df_t = pd.DataFrame(encode_day_t,
                           columns=encoder.get_feature_names_out(["Publication_Day"]))

# Concatenate with original data
df_train = pd.concat([df_train.drop("Publication_Day", axis=1), encoded_df], axis=1)
df_test = pd.concat([df_test.drop("Publication_Day", axis=1), encoded_df_t], axis=1)


In [247]:
# Label Encodeinng Episode_Sentiment

from sklearn.preprocessing import LabelEncoder
la=LabelEncoder()

df_train['Episode_Sentiment']=la.fit_transform(df_train[['Episode_Sentiment']])
df_test['Episode_Sentiment']=la.transform(df_test[['Episode_Sentiment']])

/home/pervezhasan/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/pervezhasan/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [248]:
# Ordinal Encodeing Publication_Time

from sklearn.preprocessing import OrdinalEncoder

ode=OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# ট্রেন ডেটায় ফিট করুন
ode.fit(df_train[['Publication_Time']])

# উভয় ডেটাসেট ট্রান্সফর্ম করুন
df_train['Publication_Time'] = ode.transform(df_train[['Publication_Time']])
df_test['Publication_Time'] = ode.transform(df_test[['Publication_Time']])

In [249]:
from sklearn.preprocessing import OneHotEncoder

# Initialize encoder to handle unknown categories
ohe = OneHotEncoder(drop='first',handle_unknown='ignore', sparse_output=False)

# Fit on training data
train_encoded = ohe.fit_transform(df_train[['Genre']])

# Transform both datasets
test_encoded = ohe.transform(df_test[['Genre']])

# Convert to DataFrames
train_encoded_df = pd.DataFrame(train_encoded, 
                               columns=ohe.get_feature_names_out(['Genre']))
test_encoded_df = pd.DataFrame(test_encoded,
                              columns=ohe.get_feature_names_out(['Genre']))

# Concatenate with original data
df_train = pd.concat([df_train.drop('Genre', axis=1), train_encoded_df], axis=1)
df_test = pd.concat([df_test.drop('Genre', axis=1), test_encoded_df], axis=1)


# Scaleling 

In [250]:
from sklearn.preprocessing import StandardScaler

# List of columns to scale
columns_to_scale = ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage']

# 1. Create StandardScaler instance
scaler = StandardScaler()

# 2. Fit the scaler on the selected columns of the training data
scaler.fit(df_train[columns_to_scale])

# 3. Transform the selected columns in both train and test data, replacing original columns
df_train[columns_to_scale] = scaler.transform(df_train[columns_to_scale])
df_test[columns_to_scale] = scaler.transform(df_test[columns_to_scale])

# Model Bulideing 


In [251]:
# train and test split 
from sklearn.model_selection import train_test_split

X=df_train.drop(columns=['Listening_Time_minutes'])
y=df_train['Listening_Time_minutes']

In [252]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=42)

In [253]:
# model Deployment 
from sklearn.linear_model import LinearRegression

model=LinearRegression()

model.fit(X_train,y_train) # model fit 

# মডেল ইভ্যালুয়েশন
y_pred = model.predict(X_test)


In [254]:
from sklearn.metrics import r2_score

# R² স্কোর ক্যালকুলেশন
score = r2_score(y_test, y_pred)
print(f"R² স্কোর: {score:.2f}")

# অথবা সরাসরি মডেল থেকে
model_score = model.score(X_test, y_test)
print(f"মডেল স্কোর: {model_score:.2f}")

R² স্কোর: 0.76
মডেল স্কোর: 0.76


In [255]:
final_Predict=model.predict(df_test)

In [258]:
# সাবমিশন ফাইল তৈরি করুন
submission = pd.DataFrame({
    'id':df_test['id'],
    'Listening_Time_minutes': final_Predict
})
submission.to_csv('my_submission.csv', index=False)

In [260]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

# 5-fold CV এর জন্য
cv_scores = cross_val_score(
    estimator=model,
    X=X_train,  # ফিচার ম্যাট্রিক্স
    y=y_train,  # টার্গেট ভেরিয়েবল
    cv=5,       # 5 ফোল্ড
    scoring='neg_mean_squared_error'  # স্কোরিং মেট্রিক
)

# রেজাল্ট প্রিন্ট
print("CV Scores:", -cv_scores)  # নেগেটিভ MSE কে পজিটিভ করুন
print("Mean CV MSE:", -cv_scores.mean())
print("STD CV MSE:", cv_scores.std())

CV Scores: [179.91378676 179.06898624 178.86585703 179.74645023 178.69909441]
Mean CV MSE: 179.2588349322071
STD CV MSE: 0.4838417480363118


In [261]:

from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(model,X, y, cv=5, scoring='r2')

print(f"R² Scores: {cv_scores}")
print(f"Mean R²: {cv_scores.mean():.4f}")

R² Scores: [0.754771   0.7551129  0.75839489 0.75901818 0.7568243 ]
Mean R²: 0.7568
